In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import scipy
import torch
from torch.utils.data import DataLoader

os.chdir('..')
from data_processing.datasets import PigletDataset
from neuralnet.model import SpectraMLP
from utils import beerlamb_multi
import data_processing.preprocessing as preprocessing
from config import left_cut, right_cut
from torch.utils.data import ConcatDataset
from utils import beerlamb_multi_batch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
path_to_data = "dataset/piglet_diffs/"
spectra_dataset = PigletDataset(path_to_data)
n = 15
spectra_datasets = ConcatDataset(n * [spectra_dataset])
dl = DataLoader(spectra_datasets, batch_size=n*7830, shuffle=False)
inputs, targets = next(iter(dl))

model = SpectraMLP(4)
checkpoints = torch.load('model_checkpoints/best_model.pth')
model.load_state_dict(checkpoints.state_dict())
model.to(device)
model.eval()

In [ ]:
img = scipy.io.loadmat('dataset/LWP483_10Jan2017_SharedHyperProbe.mat')
wavelengths = img['wavelengths'].astype(float)
idx = (wavelengths >= left_cut) & (wavelengths <= right_cut)
wavelengths = wavelengths[idx]

molecules, x = preprocessing.read_molecules(left_cut, right_cut, wavelengths)

In [ ]:
%%time

preds = model(inputs.float()).detach().numpy()
print(preds.shape)
#beerlamb_multi_batch(molecules, x, preds, left_cut).shape

In [ ]:
for i in range(len(preds)):
    if i % 1000 == 0:
        a = np.asarray(np.log(beerlamb_multi(molecules, x, preds[i], left_cut)))
        b = np.asarray(np.log(beerlamb_multi(molecules, x, targets[i], left_cut)))
        err = np.mean(np.abs(a - b))
        print(np.around(err, 4), preds[i], targets[i])

        plt.plot(x, np.log(beerlamb_multi(molecules, x, preds[i], left_cut)), color='b', label='Predicted')
        plt.plot(x, np.log(beerlamb_multi(molecules, x, targets[i], left_cut)), color='r', label='Real')
        plt.legend()
        plt.show()